In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

from sklearn.cluster import KMeans

csv_header = ['t0_time', 't1_time', 't2_time', 't3_time', 't4_time', 't5_time', 't6_time', 't0_glucose', 't1_glucose', 't2_glucose', 't3_glucose', 't4_glucose', 't5_glucose', 't6_glucose', 't0_slope', 't1_slope', 't2_slope', 't3_slope', 't4_slope', 't5_slope', 't6_slope', 't0_iob', 't1_iob', 't2_iob', 't3_iob', 't4_iob', 't5_iob', 't6_iob', 't0_mob', 't1_mob', 't2_mob', 't3_mob', 't4_mob', 't5_mob', 't6_mob', 't0_morning', 't1_morning', 't2_morning', 't3_morning', 't4_morning', 't5_morning', 't6_morning', 't0_afternoon', 't1_afternoon', 't2_afternoon', 't3_afternoon', 't4_afternoon', 't5_afternoon', 't6_afternoon', 't0_evening', 't1_evening', 't2_evening', 't3_evening', 't4_evening', 't5_evening', 't6_evening', 't0_night', 't1_night', 't2_night', 't3_night', 't4_night', 't5_night', 't6_night']

In [2]:
s2_data = pd.read_csv('data/subject2_instances.csv', header=None, names=csv_header)

In [3]:
rows, features = s2_data.shape
print "Rows: {}, Features: {}".format(rows,features)

Rows: 2663, Features: 63
